In [1]:
import os
import concurrent.futures
import multiprocess as mp
import random


from astropy.io import fits
from matplotlib import pyplot as plt
import numpy as np
import lightkurve as lk
from astroquery.simbad import Simbad
import pandas as pd 
from numpy import genfromtxt


#tf
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [2]:
hdul = fits.open('/Users/michaelsong/Documents/GitHub/starzam/Labels/J_ApJS_236_42_table2.dat.gz.fits')

hdul.info()
print(hdul[1].data)

Filename: /Users/michaelsong/Documents/GitHub/starzam/Labels/J_ApJS_236_42_table2.dat.gz.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     114   ()      
  1  table2.dat    1 TableHDU       162   16094R x 20C   [I8, I4, I3, F5.3, F5.3, F5.2, F4.2, F4.2, F4.2, F5.2, F4.2, F4.2, F4.2, F5.2, F4.2, F4.2, F4.2, F5.2, F5.2, I1]   


[(757137, 4751, 139, 2.383, 0.012, -0.08, 0.3, 1.7, 0.14, 13.91, 0.41, 1.55, 0.13, 13.26, 0.39, 1.72, 0.14, 13.97, 0.41, 1)
 (892010, 4834, 151, 2.161, 0.023, 0.21, 0.3, 1.42, 0.29, 16.37, 1.38, 1.36, 0.28, 16.02, 1.34, 1.43, 0.3, 16.47, 1.39, 0)
 (892738, 4534, 135, 1.769, 0.022, -0.25, 0.3, 1.15, 0.21, 23.15, 1.62, 0.99, 0.18, 21.49, 1.48, 1.12, 0.2, 22.86, 1.6, 0)
 ...
 (12885196, 4846, 142, 2.878, 0.008, 0.07, 0.3, 1.55, 0.1, 7.5, 0.17, 1.46, 0.09, 7.29, 0.16, 1.57, 0.1, 7.55, 0.17, 1)
 (12885373, 4855, 144, 2.627, 0.009, 0.12, 0.3, 1.49, 0.09, 9.83, 0.22, 1.42, 0.09, 9.59, 0.22, 1.51, 0.1, 9.9, 0.22, 1)
 (12934574, 4834, 143, 2.387, 0.012, 0.01, 0.3, 1.02, 0.1, 10.72, 0.42, 0.99, 0.1, 10.57, 0.41, 1.03, 0.1, 10.74, 0.42, 2)]


In [3]:
print(hdul[1].data)
teffe = []
for i in range(len(hdul[1].data)):
   teffe = np.append(teffe, hdul[1].data[i][2])
print(teffe)

[(757137, 4751, 139, 2.383, 0.012, -0.08, 0.3, 1.7, 0.14, 13.91, 0.41, 1.55, 0.13, 13.26, 0.39, 1.72, 0.14, 13.97, 0.41, 1)
 (892010, 4834, 151, 2.161, 0.023, 0.21, 0.3, 1.42, 0.29, 16.37, 1.38, 1.36, 0.28, 16.02, 1.34, 1.43, 0.3, 16.47, 1.39, 0)
 (892738, 4534, 135, 1.769, 0.022, -0.25, 0.3, 1.15, 0.21, 23.15, 1.62, 0.99, 0.18, 21.49, 1.48, 1.12, 0.2, 22.86, 1.6, 0)
 ...
 (12885196, 4846, 142, 2.878, 0.008, 0.07, 0.3, 1.55, 0.1, 7.5, 0.17, 1.46, 0.09, 7.29, 0.16, 1.57, 0.1, 7.55, 0.17, 1)
 (12885373, 4855, 144, 2.627, 0.009, 0.12, 0.3, 1.49, 0.09, 9.83, 0.22, 1.42, 0.09, 9.59, 0.22, 1.51, 0.1, 9.9, 0.22, 1)
 (12934574, 4834, 143, 2.387, 0.012, 0.01, 0.3, 1.02, 0.1, 10.72, 0.42, 0.99, 0.1, 10.57, 0.41, 1.03, 0.1, 10.74, 0.42, 2)]
[139. 151. 135. ... 142. 144. 143.]


In [4]:
print(sum(teffe)/len(teffe))
count = 0
for i in range(len(teffe)):
    if teffe[i] <= 100:
        count += 1
print(count, len(teffe)-count)

122.85609543929415
7625 8469


In [5]:
test_lk = lk.search_lightcurve('KIC '+str(757137), author='Kepler',exptime=1800)[2].download().remove_outliers(sigma=20, sigma_upper=4).normalize()
print(len(test_lk.time.value))
l = random.randrange(250, len(test_lk.time.value))
s = random.randrange(0,len(test_lk.time.value)-l)
print(s,l)
test_lk2 = test_lk.truncate(after=test_lk.time.value[s+l],before=test_lk.time.value[s])
test_periodogram = test_lk2.to_periodogram(frequency = [x for x in np.arange(1,24.5,0.1)])
freq = np.asarray([test_periodogram.frequency,test_periodogram.power*100000])
print(freq[1])

4134
770 1738
[5.24204169 6.3253993  2.81822677 4.36254768 5.02275538 1.64259439
 8.23901335 3.80093387 3.47781742 4.67749315 5.94100463 4.21183629
 8.79729259 2.56233368 2.65508444 2.48662571 1.35904076 2.92738102
 3.50587931 6.12230787 3.33885046 4.26613244 0.80785171 1.63094469
 4.10210314 2.38671387 3.09196283 1.84297081 3.97728383 2.59121494
 2.29998895 1.0885984  0.88510526 0.4453222  1.73753853 0.18103211
 0.43585955 1.68101277 1.0636674  1.85000617 2.6784093  0.4572161
 1.55523051 1.58822179 0.97404079 1.76364973 1.94745304 0.62560543
 0.46727238 1.49189682 0.58807248 0.91329129 1.31698368 0.48116855
 1.13067258 0.78262293 0.89455732 1.23372847 0.51191291 1.32041496
 0.73435041 0.55920519 1.16726627 0.17746462 1.33527485 0.79465399
 0.69607481 0.15598112 0.78855346 0.83793982 0.45647425 0.79817084
 0.89986127 0.41535298 0.91136554 0.4510925  0.33310507 0.65072494
 0.29556039 0.63622895 0.21631519 0.18728782 0.22235355 0.59312701
 0.38025965 0.3755345  0.21607905 0.64028148 0.23

In [32]:
def prepsr(i,sr,j):
    r = True
    if r:
        test_lk = sr[j].download().remove_outliers(sigma=20, sigma_upper=4).normalize()
        # l = random.randrange(i+240, len(test_lk.time.value))
        # s = random.randrange(0,len(test_lk.time.value)-l)
        lightcurve = test_lk.truncate(after=test_lk.time.value[1+i],before=test_lk.time.value[0])
    else:
        lightcurve = sr[j].download().remove_outliers(sigma=20, sigma_upper=4).normalize()
    
    periodogram = lightcurve.to_periodogram(frequency = [x for x in np.arange(1,24.5,0.1)])
    # periodogram = periodogram.bin(binsize=10, method='mean')
    freq = np.asarray([periodogram.frequency,periodogram.power*100000])
    # pd.DataFrame(freq[1]).to_csv("/content/drive/MyDrive/Research - Yihong Song and Rohit Prasanna/Yihong/Data/"+str(labeled_data["Kepler_ID"][i])+"_"+str(j)+".csv")
    return np.append(np.append(hdul[1].data[i][0:7],[j]),freq[1])

def getlc(i):
    lc_data = []
    sr = lk.search_lightcurve('KIC '+str(hdul[1].data[i][0]), author='Kepler',exptime=1800)
    # return prepsr(i,sr,1)
    for j in range(len(sr)):
        # x = wrap(prepsr(i,sr,j))
        # if x is not None:
        lc_data.append(prepsr(i,sr,j))
        # y = wrap(external_function_raises)
        # if y is not None:
        #     lc_data.append(y)
    return lc_data

def getdata(start):
    # for k in range(start,int(191449/100)+1,1):
    for k in range(start,start+1,1):
        lc_data = []
        for i in range(k*10,(k+1)*10,1):
            try:
                lc_data = lc_data+getlc(i)
            except:
                continue
            print("\r"+str(i),end="")
        # print(lc_data)
        pd.DataFrame(lc_data).to_csv("/Users/michaelsong/Documents/GitHub/starzam/Data_Length/"+str(k)+".csv")

In [33]:
getdata(0)

/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super(

0

/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


1

/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super(

...

/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


8

/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super(

9

/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


In [34]:
# with concurrent.futures.ProcessPoolExecutor() as executor:
#     print(executor.map(getdata, [1,2,3]))

with mp.Pool(8) as pool:
    print(pool.map(getdata, [x for x in range(0,100)]))

/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super(

0

/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


1

/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


...

215

/Users/michaelsong/miniforge3/lib/python3.9/site-packages/astropy/units/quantity.py:614: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


74

335

917[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
